In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_products = pd.read_csv("/kaggle/input/learnplatform-covid19-impact-on-digital-learning/products_info.csv")
df_districts = pd.read_csv("/kaggle/input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv")

In [ ]:
df_products.head()

In [ ]:
df_districts.head()

In [ ]:
df_engagements = pd.DataFrame()
for did in df_districts["district_id"]:
    temp_df = pd.read_csv(f'/kaggle/input/learnplatform-covid19-impact-on-digital-learning/engagement_data/{did}.csv')
    temp_df["district_id"] = did
    df_engagements = pd.concat([df_engagements, temp_df])
df_engagements.head()

In [ ]:
df_engagements.info()

In [ ]:
df_engagements[df_engagements["time"] == "2020-01-01"]["time"]

In [ ]:
df_engagements["time"].value_counts()

In [ ]:
df_engagements["engagement_index"].value_counts()

In [ ]:
df_engagements["lp_id"].value_counts()

In [ ]:
df_engagements[df_engagements["lp_id"] == 95731].value_counts()

In [ ]:
df_engagements

In [ ]:
df_products["Product Name"].value_counts()

In [ ]:
df_products["Primary Essential Function"].value_counts()

In [ ]:
df_products["Sector(s)"].value_counts()

# Q1 & Q2: Digital Conn & Engagement in 2020 and the Trends for Future

In [ ]:
temp = df_engagements.groupby(["lp_id","district_id","time"]).sum().reset_index()
temp.head()

In [ ]:
temp["lp_id"] = temp["lp_id"].astype("int")
temp.head()

In [ ]:
year = []
month = []
day = []
for t in temp["time"]:
    z = t.split("-")
    year.append(z[0])
    month.append(z[1])
    day.append(z[2])
temp["year"] = year
temp["month"] = month
temp["day"] = day
temp.head()

In [ ]:
df_general_pic = temp[["month", "engagement_index", "pct_access"]].groupby(["month"]).sum().reset_index()

from sklearn.preprocessing import StandardScaler, MinMaxScaler

ss = MinMaxScaler()
df_general_pic["engagement_index"] = ss.fit_transform(np.expand_dims(df_general_pic["engagement_index"], axis=1))
df_general_pic["pct_access"] = ss.fit_transform(np.expand_dims(df_general_pic["pct_access"], axis=1))

df_general_pic.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# plot lines
plt.plot(df_general_pic["month"], df_general_pic["engagement_index"], label="Engagement")
plt.plot(df_general_pic["month"], df_general_pic["pct_access"], label="Access")
plt.legend()
plt.show()

In [ ]:
df_general_pic_each_month = temp[["month", "day", "engagement_index", "pct_access"]].groupby(["month", "day"]).sum().reset_index()

from sklearn.preprocessing import StandardScaler, MinMaxScaler

ss = MinMaxScaler()
df_general_pic_each_month["engagement_index"] = ss.fit_transform(np.expand_dims(df_general_pic_each_month["engagement_index"], axis=1))
df_general_pic_each_month["pct_access"] = ss.fit_transform(np.expand_dims(df_general_pic_each_month["pct_access"], axis=1))

df_general_pic_each_month.head()

In [ ]:
for mo in df_general_pic_each_month["month"].unique():
    df_z = df_general_pic_each_month[df_general_pic_each_month["month"] == mo]
    # plot lines
    plt.figure(figsize=(10,5))
    plt.plot(df_z["day"], df_z["engagement_index"], label=f"Engagement-{mo}")
    plt.plot(df_z["day"], df_z["pct_access"], label=f"Access-{mo}")
    plt.title(f"Pic-{mo}")
    plt.legend()
    plt.show()

# Q3: Engagement with Different EduTech

In [ ]:
df_engagement_edutech = temp[["lp_id", "pct_access", "engagement_index"]].groupby("lp_id").sum().reset_index()

from sklearn.preprocessing import MinMaxScaler

ss = MinMaxScaler()
df_engagement_edutech["engagement_index"] = ss.fit_transform(np.expand_dims(df_engagement_edutech["engagement_index"], axis=1))
df_engagement_edutech["pct_access"] = ss.fit_transform(np.expand_dims(df_engagement_edutech["pct_access"], axis=1))

df_engagement_edutech.head()

In [ ]:
df_engagement_edutech["lp_id"].unique()

In [ ]:
df_engagement_edutech["engagement_index"].min(), df_engagement_edutech["engagement_index"].max()

In [ ]:
import seaborn as sns

plt.figure(figsize=(20,5))
sns.scatterplot(df_engagement_edutech["lp_id"], df_engagement_edutech["engagement_index"])
plt.show()

In [ ]:
top10_edutech_eg = df_engagement_edutech[(df_engagement_edutech["engagement_index"] > 0.1) & (df_engagement_edutech["engagement_index"] <= 1)]
top10_edutech_eg

In [ ]:
top10_edutech_eg_prod = pd.merge(top10_edutech_eg, df_products, left_on="lp_id", right_on="LP ID")
top10_edutech_eg_prod.sort_values(by="engagement_index", ascending=False)

In [ ]:
import seaborn as sns

plt.figure(figsize=(20,5))
sns.scatterplot(df_engagement_edutech["lp_id"], df_engagement_edutech["pct_access"])
plt.show()

In [ ]:
top10_edutech_ac = df_engagement_edutech[(df_engagement_edutech["pct_access"] > 0.1) & (df_engagement_edutech["pct_access"] <= 1)]
top10_edutech_ac

In [ ]:
top10_edutech_ac_prod = pd.merge(top10_edutech_ac, df_products, left_on="lp_id", right_on="LP ID")
top10_edutech_ac_prod.sort_values(by="pct_access", ascending=False)

## Products df
* LP ID = Product Name

In [ ]:
df_products_engagements = pd.merge(left=df_products, right=df_engagements, left_on='LP ID', right_on='lp_id')
df_products_engagements.head()

In [ ]:
df_sector_engagements = df_products_engagements[["Sector(s)", "pct_access", "engagement_index"]]
df_sector_engagements = df_sector_engagements.groupby(['Sector(s)']).sum().reset_index()

from sklearn.preprocessing import StandardScaler, MinMaxScaler

ss = MinMaxScaler()
df_sector_engagements["engagement_index"] = ss.fit_transform(np.expand_dims(df_sector_engagements["engagement_index"], axis=1))
df_sector_engagements["pct_access"] = ss.fit_transform(np.expand_dims(df_sector_engagements["pct_access"], axis=1))

df_sector_engagements.head()

In [ ]:
plt.figure(figsize=(15,5))
plt.bar(df_sector_engagements["Sector(s)"], df_sector_engagements["pct_access"], color='r')
plt.bar(df_sector_engagements["Sector(s)"], df_sector_engagements["engagement_index"], bottom=df_sector_engagements["pct_access"], color='b')
plt.show()

In [ ]:
df_ess_engagements = df_products_engagements[["Primary Essential Function", "pct_access", "engagement_index"]]
df_ess_engagements = df_ess_engagements.groupby(['Primary Essential Function']).sum().reset_index()

from sklearn.preprocessing import StandardScaler, MinMaxScaler

ss = MinMaxScaler()
df_ess_engagements["engagement_index"] = ss.fit_transform(np.expand_dims(df_ess_engagements["engagement_index"], axis=1))
df_ess_engagements["pct_access"] = ss.fit_transform(np.expand_dims(df_ess_engagements["pct_access"], axis=1))

df_ess_engagements.head()

In [ ]:
plt.figure(figsize=(20,5))
plt.bar(df_ess_engagements["Primary Essential Function"], df_ess_engagements["pct_access"], color='r')
plt.bar(df_ess_engagements["Primary Essential Function"], df_ess_engagements["engagement_index"], bottom=df_ess_engagements["pct_access"], color='b')
plt.xticks(rotation=90)
plt.show()

# Q4: Engagement with Different Districts

In [ ]:
temp.head()

In [ ]:
df_districts.head()

In [ ]:
df_culture_engagements = pd.merge(temp[["district_id", "pct_access", "engagement_index"]], df_districts[["district_id", "pct_black/hispanic", "pct_free/reduced", "pp_total_raw"]], on=("district_id"))
df_culture_engagements

In [ ]:
black_eg = df_culture_engagements[["pct_black/hispanic", "pct_access", "engagement_index"]].groupby("pct_black/hispanic").sum().reset_index()

from sklearn.preprocessing import MinMaxScaler

ss = MinMaxScaler()
black_eg["engagement_index"] = ss.fit_transform(np.expand_dims(black_eg["engagement_index"], axis=1))
black_eg["pct_access"] = ss.fit_transform(np.expand_dims(black_eg["pct_access"], axis=1))

black_eg.head()

In [ ]:
plt.figure(figsize=(15,5))
plt.bar(black_eg["pct_black/hispanic"], black_eg["pct_access"], color='r')
plt.bar(black_eg["pct_black/hispanic"], black_eg["engagement_index"], bottom=black_eg["pct_access"], color='b')
plt.show()

In [ ]:
free_eg = df_culture_engagements[["pct_free/reduced", "pct_access", "engagement_index"]].groupby("pct_free/reduced").sum().reset_index()

from sklearn.preprocessing import MinMaxScaler

ss = MinMaxScaler()
free_eg["engagement_index"] = ss.fit_transform(np.expand_dims(free_eg["engagement_index"], axis=1))
free_eg["pct_access"] = ss.fit_transform(np.expand_dims(free_eg["pct_access"], axis=1))

free_eg.head()

In [ ]:
plt.figure(figsize=(15,5))
plt.bar(free_eg["pct_free/reduced"], free_eg["pct_access"], color='r')
plt.bar(free_eg["pct_free/reduced"], free_eg["engagement_index"], bottom=free_eg["pct_access"], color='b')
plt.show()

In [ ]:
pp_total_raw = df_culture_engagements[["pp_total_raw", "pct_access", "engagement_index"]].groupby("pp_total_raw").sum().reset_index()

from sklearn.preprocessing import MinMaxScaler

ss = MinMaxScaler()
pp_total_raw["engagement_index"] = ss.fit_transform(np.expand_dims(pp_total_raw["engagement_index"], axis=1))
pp_total_raw["pct_access"] = ss.fit_transform(np.expand_dims(pp_total_raw["pct_access"], axis=1))

pp_total_raw.head()

In [ ]:
plt.figure(figsize=(15,5))
plt.bar(pp_total_raw["pp_total_raw"], pp_total_raw["pct_access"], color='r')
plt.bar(pp_total_raw["pp_total_raw"], pp_total_raw["engagement_index"], bottom=pp_total_raw["pct_access"], color='b')
plt.xticks(rotation=45)
plt.show()

In [ ]:
df_engagement_district = pd.merge(temp, df_districts[["district_id", "state", "locale"]], on=["district_id"])
df_engagement_district = df_engagement_district.groupby(["state","locale","time"]).sum().reset_index()
df_engagement_district.head()

In [ ]:
df_engagement_district.head(20)

In [ ]:
df_engagement_district_sub = df_engagement_district[["state", "pct_access", "engagement_index"]].groupby(["state"]).sum().reset_index()

from sklearn.preprocessing import StandardScaler, MinMaxScaler

ss = MinMaxScaler()
df_engagement_district_sub["engagement_index"] = ss.fit_transform(np.expand_dims(df_engagement_district_sub["engagement_index"], axis=1))
df_engagement_district_sub["pct_access"] = ss.fit_transform(np.expand_dims(df_engagement_district_sub["pct_access"], axis=1))

df_engagement_district_sub.head()

In [ ]:
import seaborn as sns

plt.figure(figsize=(20,5))
sns.barplot(df_engagement_district_sub["engagement_index"], df_engagement_district_sub["state"])
plt.show()

In [ ]:
import seaborn as sns

plt.figure(figsize=(20,5))
sns.barplot(df_engagement_district_sub["pct_access"], df_engagement_district_sub["state"])
plt.show()

In [ ]:
import seaborn as sns

plt.figure(figsize=(20,5))
sns.scatterplot(df_engagement_district["district_id"], df_engagement_district["engagement_index"])
plt.show()